<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/transientcluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

     |████████████████████████████████| 131 kB 7.6 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 8.5 MB 41.3 MB/s 
     |████████████████████████████████| 138 kB 76.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
import json;
import boto3;
client = boto3.client('emr', region_name='us-east-2',aws_access_key_id='',aws_secret_access_key='')


In [3]:
event={'file_name':'iris.csv','bucket_name':'sourcedemotesingset'}

In [9]:
def lambda_handler(event, context):
    file_name = event['file_name']
    bucketName=event['bucket_name']
    print("File Name : ",file_name)
    print("Bucket Name : ",bucketName)
    backend_code="{}"
    spark_submit = [
    'spark-submit',
    '--master', 'yarn',
    '--deploy-mode', 'cluster',
    backend_code,
    bucketName,
    file_name
    ]
    print("Spark Submit : ",spark_submit)
    cluster_id = client.run_job_flow(
    Name="transient_demo_testing",
    Instances={
    'InstanceGroups': [
    {
    'Name': "Master",
    'Market': 'ON_DEMAND',
    'InstanceRole': 'MASTER',
    'InstanceType': 'm5.xlarge',
    'InstanceCount': 1,
    },
    {
    'Name': "Slave",
    'Market': 'ON_DEMAND',
    'InstanceRole': 'CORE',
    'InstanceType': 'm5.xlarge',
    'InstanceCount': 2,
    }
    ],
    'Ec2KeyName': 'helloworld',
    'KeepJobFlowAliveWhenNoSteps': False,
    'TerminationProtected': False,
    'Ec2SubnetId': 'subnet-{}',
    },
    LogUri="s3://aws-logs-194680882756-us-east-2/elasticmapreduce/",
    ReleaseLabel= '{}',
    Steps=[{"Name": "testJobGURU",
    'ActionOnFailure': 'CONTINUE',
    'HadoopJarStep': {
    'Jar': 'command-runner.jar',
    'Args': spark_submit
    }
    }],
    BootstrapActions=[],
    VisibleToAllUsers=True,
    JobFlowRole="EMR_EC2_DefaultRole",
    ServiceRole="EMR_DefaultRole",
    Applications = [ {'Name': 'Spark'},{'Name':'Hive'}])
    print(cluster_id)

In [10]:
lambda_handler(event, 0)

File Name :  iris.csv
Bucket Name :  sourcedemotesingset
Spark Submit :  ['spark-submit', '--master', 'yarn', '--deploy-mode', 'cluster', 's3://codetestingsetdemo/sparkdemo.py', 'sourcedemotesingset', 'iris.csv']
{'JobFlowId': 'j-1E0I1EQJ4WZTD', 'ClusterArn': 'arn:aws:elasticmapreduce:us-east-2:194680882756:cluster/j-1E0I1EQJ4WZTD', 'ResponseMetadata': {'RequestId': 'c80b327f-0287-4941-b3f6-dc6ca19aeef2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c80b327f-0287-4941-b3f6-dc6ca19aeef2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '118', 'date': 'Mon, 17 Jan 2022 15:54:55 GMT'}, 'RetryAttempts': 0}}
